In [35]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

df = pd.read_csv("synthetic_dataset.csv")

## Estimación de Parámetros con OLS

In [36]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,Y
0,4.967142,-1.382643,NaN,15.230299,-2.341534,-2.341370,15.792128,7.674347,-4.694744,5.425600,-4.634177,-4.657298,2.419623,-19.132802,-17.249178,152.273643
1,-5.622875,-10.128311,3.142473,-9.080241,-14.123037,14.656488,-2.257763,NaN,-14.247482,-5.443827,1.109226,-11.509936,3.756980,-6.006387,-2.916937,144.947630
2,-6.017066,18.522782,-0.134972,-10.577109,8.225449,NaN,2.088636,-19.596701,-13.281860,1.968612,7.384666,1.713683,-1.156483,-3.011037,-14.785220,188.449076
3,-7.198442,-4.606388,NaN,3.436183,-17.630402,3.240840,-3.850823,-6.769220,6.116763,10.309995,9.312801,NaN,-3.092124,3.312634,9.755451,107.875692
4,-4.791742,-1.856590,-11.063350,-11.962066,8.125258,13.562400,-0.720101,10.035329,3.616360,-6.451198,3.613956,15.380366,-0.358260,15.646437,-26.197451,-78.724744


In [37]:
print(df.isnull().sum())

X1     205
X2     202
X3     210
X4     222
X5     210
X6     195
X7     185
X8     193
X9     195
X10    195
X11    189
X12    198
X13    211
X14    198
X15    192
Y        0
dtype: int64


In [38]:
total_datos = 4000 
print(f"Total de datos: {total_datos}")

X1 = (205 / total_datos) * 100
X2 = (202 / total_datos) * 100
X3 = (210 / total_datos) * 100
X4 = (222 / total_datos) * 100


print(f"El porcentaje que representan de datos nulos en cada una de las categorías son los siguientes:\n"
      f"X1: {X1:.2f}%\n"
      f"X2: {X2:.2f}%\n"
      f"X3: {X3:.2f}%\n"
      f"X4: {X4:.2f}%\n"
)

Total de datos: 4000
El porcentaje que representan de datos nulos en cada una de las categorías son los siguientes:
X1: 5.12%
X2: 5.05%
X3: 5.25%
X4: 5.55%



## Normalización

In [39]:
numerical_cols = np.array(["X1", "X2", "X3", "X4"])
for col in numerical_cols:  # 'col' ya contiene el nombre de la columna
    if col in df.columns:  # Verificar que la columna existe en df o
        stat, p_value = stats.normaltest(df[col].dropna())  
        print(f" {col}: Estadístico = {stat}, p-valor = {p_value}")

        if p_value < 0.05:
            print(" Los datos NO siguen una distribución normal.\n")
        else:
            print(" Los datos siguen una distribución normal.\n")
    else:
        print(f" La columna '{col}' no existe en el DataFrame.\n")

 X1: Estadístico = 3.165120884485229, p-valor = 0.20544838545904986
 Los datos siguen una distribución normal.

 X2: Estadístico = 1.107147111082594, p-valor = 0.5748917277151018
 Los datos siguen una distribución normal.

 X3: Estadístico = 0.8312062793277601, p-valor = 0.6599421233743414
 Los datos siguen una distribución normal.

 X4: Estadístico = 1.0703099682544395, p-valor = 0.5855785277345695
 Los datos siguen una distribución normal.



Si el porcentaje de valores faltantes es 5%, lo ideal es imputarlos en lugar de eliminarlos para no perder información. En este caso, utilizaremos la imputación con la media, ya que es una estrategia efectiva cuando los datos siguen una distribución aproximadamente normal y no presentan valores atípicos significativos.

Para imputar los valores faltantes con la media en un DataFrame de pandas, puedes usar el siguiente código:

In [8]:
df.fillna(df.mean(), inplace=True)
print(df.isnull().sum())


X1     0
X2     0
X3     0
X4     0
X5     0
X6     0
X7     0
X8     0
X9     0
X10    0
X11    0
X12    0
X13    0
X14    0
X15    0
Y      0
dtype: int64


In [40]:
import numpy as np
import pandas as pd

# Simulación de datos con valores faltantes
np.random.seed(42)
total_datos = 4000
n_features = 4  # Ahora hay 4 variables independientes

# Variables independientes (X) con valores faltantes
X = np.random.rand(total_datos, n_features)
X[np.random.choice(total_datos, 205), 0] = np.nan  
X[np.random.choice(total_datos, 202), 1] = np.nan  
X[np.random.choice(total_datos, 210), 2] = np.nan  
X[np.random.choice(total_datos, 222), 3] = np.nan  

y = 3.5 + 2.3 * X[:, 0] + 1.7 * X[:, 1] + 0.9 * X[:, 2] + 1.5 * X[:, 3] + np.random.randn(total_datos)

df = pd.DataFrame(X, columns=["X1", "X2", "X3", "X4"])
df["y"] = y

df.fillna(df.mean(), inplace=True)

X = df[["X1", "X2", "X3", "X4"]].values
y = df["y"].values

# Agregar columna de unos para el término independiente (β0)
X_b = np.c_[np.ones((X.shape[0], 1)), X]  # Añade una columna de unos a X

# 3️⃣ Cálculo de los coeficientes usando la fórmula OLS: β = (XᵀX)⁻¹ Xᵀy
XT_X = X_b.T @ X_b  # Multiplicación de la transpuesta de X por X
XT_X_inv = np.linalg.inv(XT_X)  # Inversa de (XᵀX)
XT_y = X_b.T @ y  # Multiplicación de la transpuesta de X por y
beta = XT_X_inv @ XT_y  # Cálculo final de los coeficientes

print(f"Coeficientes estimados (β):\n")
print(f"β0 (Intercepto): {beta[0]:.4f}")
print(f"β1 (X1): {beta[1]:.4f}")
print(f"β2 (X2): {beta[2]:.4f}")
print(f"β3 (X3): {beta[3]:.4f}")
print(f"β4 (X4): {beta[4]:.4f}")


Coeficientes estimados (β):

β0 (Intercepto): 4.0499
β1 (X1): 1.9369
β2 (X2): 1.4484
β3 (X3): 0.7547
β4 (X4): 1.2067


## Numpy